Persistence:Checkpointer and Threads

In [10]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
    

In [11]:
load_dotenv()

llm = ChatGroq( model="llama-3.3-70b-versatile")

In [12]:
llm.invoke("what is TCS?")

AIMessage(content="TCS can refer to different things depending on the context. Here are a few possible meanings:\n\n1. **Tata Consultancy Services**: TCS is a multinational information technology (IT) consulting and business solutions company headquartered in Mumbai, India. It is one of the largest IT companies in the world and provides a wide range of services, including consulting, digital transformation, and IT outsourcing.\n2. **Technical Customer Support**: In some cases, TCS can also refer to Technical Customer Support, which is a team or department responsible for providing technical assistance and support to customers, often for a company's products or services.\n3. **Transportation and Classification System**: In logistics and supply chain management, TCS can refer to a Transportation and Classification System, which is a system used to manage and classify transportation modes, routes, and carriers.\n4. **Thermal Control System**: In engineering and technology, TCS can refer t

In [13]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [14]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [15]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [16]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver() #just give a check pointer object

workflow = graph.compile(checkpointer=checkpointer)

In [17]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1) # just mention the config details

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)',
 'explanation': 'The joke relies on a play on words to create humor. The setup "Why was the pizza in a bad mood?" primes the listener to expect a reason related to the pizza\'s emotional state. The punchline "Because it was feeling a little crusty!" subverts this expectation by using the word "crusty" in a double sense.\n\nIn one sense, "crusty" can describe someone or something that is irritable, gruff, or in a bad mood. However, in the context of a pizza, "crusty" also refers to the crispy, golden-brown edge of the pizza, which is a defining characteristic of the food.\n\nThe joke exploits this dual meaning of "crusty" to create a humorous connection between the pizza\'s emotional state and its physical properties. The phrase "feeling a little crusty" is a clever play on words that bridges the gap between the pizza\'s mood and its crust, creating a lighthearted and amu

In [18]:
workflow.get_state(config1) #this heplps to get final value of the state.

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)', 'explanation': 'The joke relies on a play on words to create humor. The setup "Why was the pizza in a bad mood?" primes the listener to expect a reason related to the pizza\'s emotional state. The punchline "Because it was feeling a little crusty!" subverts this expectation by using the word "crusty" in a double sense.\n\nIn one sense, "crusty" can describe someone or something that is irritable, gruff, or in a bad mood. However, in the context of a pizza, "crusty" also refers to the crispy, golden-brown edge of the pizza, which is a defining characteristic of the food.\n\nThe joke exploits this dual meaning of "crusty" to create a humorous connection between the pizza\'s emotional state and its physical properties. The phrase "feeling a little crusty" is a clever play on words that bridges the gap between the pizza\'s mood and its crust, creating a l

In [19]:
list(workflow.get_state_history(config1)) #helps to display whole history not just final state

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)', 'explanation': 'The joke relies on a play on words to create humor. The setup "Why was the pizza in a bad mood?" primes the listener to expect a reason related to the pizza\'s emotional state. The punchline "Because it was feeling a little crusty!" subverts this expectation by using the word "crusty" in a double sense.\n\nIn one sense, "crusty" can describe someone or something that is irritable, gruff, or in a bad mood. However, in the context of a pizza, "crusty" also refers to the crispy, golden-brown edge of the pizza, which is a defining characteristic of the food.\n\nThe joke exploits this dual meaning of "crusty" to create a humorous connection between the pizza\'s emotional state and its physical properties. The phrase "feeling a little crusty" is a clever play on words that bridges the gap between the pizza\'s mood and its crust, creating a 

In [20]:
#for another thread id:--
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a lifelong commitment.',
 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of the explanation:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the listener to expect a reason related to relationships or commitment. The punchline "Because it was afraid of getting tangled up in a lifelong commitment" is where the wordplay comes in.\n\nThe phrase "tangled up" has a double meaning here:\n\n1. **Literal meaning**: Spaghetti is a type of long, thin, flexible pasta that can easily become tangled or knotted. So, in a literal sense, spaghetti can get physically tangled up.\n2. **Figurative meaning**: In the context of relationships, "tangled up" is an idiomatic expression that means to become deeply involved or entangled in a complicated situation, often implying a sense of being

In [21]:
workflow.get_state(config2) #provide current step state values

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a lifelong commitment.', 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of the explanation:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the listener to expect a reason related to relationships or commitment. The punchline "Because it was afraid of getting tangled up in a lifelong commitment" is where the wordplay comes in.\n\nThe phrase "tangled up" has a double meaning here:\n\n1. **Literal meaning**: Spaghetti is a type of long, thin, flexible pasta that can easily become tangled or knotted. So, in a literal sense, spaghetti can get physically tangled up.\n2. **Figurative meaning**: In the context of relationships, "tangled up" is an idiomatic expression that means to become deeply involved or entangled in a complicated situation, often implyi

In [22]:
list(workflow.get_state_history(config2)) #provide each steps state values

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a lifelong commitment.', 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of the explanation:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the listener to expect a reason related to relationships or commitment. The punchline "Because it was afraid of getting tangled up in a lifelong commitment" is where the wordplay comes in.\n\nThe phrase "tangled up" has a double meaning here:\n\n1. **Literal meaning**: Spaghetti is a type of long, thin, flexible pasta that can easily become tangled or knotted. So, in a literal sense, spaghetti can get physically tangled up.\n2. **Figurative meaning**: In the context of relationships, "tangled up" is an idiomatic expression that means to become deeply involved or entangled in a complicated situation, often imply

#### Fault Tolerance

In [3]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [4]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [5]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [6]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

#### Time Travel :basically this is used to debugging purpose and executing intermediate steps..

In [26]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06e56f-66bf-60b6-8000-75740705f3f6"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06e56f-66bf-60b6-8000-75740705f3f6'}}, metadata={'source': 'loop', 'writes': None, 'thread_id': '1', 'step': 0, 'parents': {}}, created_at='2025-07-31T21:40:22.899730+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e56f-66b7-6ad2-bfff-3ed4d56d714b'}}, tasks=(PregelTask(id='83418412-f856-f280-de40-dd2aed66ecfe', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)'}),))

In [27]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06e56f-66bf-60b6-8000-75740705f3f6"}})

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)',
 'explanation': 'A classic play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it works:\n\nThe joke starts by setting up a situation where a pizza is in a bad mood, which is an unexpected and anthropomorphic concept, as pizzas are inanimate objects and can\'t actually have emotions. The listener is then prompted to wonder why the pizza might be in a bad mood.\n\nThe punchline, "Because it was feeling a little crusty!" is where the wordplay comes in. "Crusty" has a double meaning here:\n\n1. In terms of personality, "crusty" can describe someone who is irritable, gruff, or short-tempered. So, in this context, the pizza is "feeling a little crusty" because it\'s in a bad mood.\n2. However, "crusty" also refers to the outer layer of a pizza, which is crispy and crunchy. This is a clever play on words, as the joke is making a connection betw

In [28]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)', 'explanation': 'A classic play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it works:\n\nThe joke starts by setting up a situation where a pizza is in a bad mood, which is an unexpected and anthropomorphic concept, as pizzas are inanimate objects and can\'t actually have emotions. The listener is then prompted to wonder why the pizza might be in a bad mood.\n\nThe punchline, "Because it was feeling a little crusty!" is where the wordplay comes in. "Crusty" has a double meaning here:\n\n1. In terms of personality, "crusty" can describe someone who is irritable, gruff, or short-tempered. So, in this context, the pizza is "feeling a little crusty" because it\'s in a bad mood.\n2. However, "crusty" also refers to the outer layer of a pizza, which is crispy and crunchy. This is a clever play on words, as the joke is maki

In [29]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06e56f-66b7-6ad2-bfff-3ed4d56d714b", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06e591-188b-6ea2-8000-c15669419e53'}}

In [30]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e591-188b-6ea2-8000-c15669419e53'}}, metadata={'source': 'update', 'writes': {'__start__': {'topic': 'samosa'}}, 'thread_id': '1', 'step': 0, 'parents': {}, 'checkpoint_id': '1f06e56f-66b7-6ad2-bfff-3ed4d56d714b', 'checkpoint_ns': ''}, created_at='2025-07-31T21:55:27.380445+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06e56f-66b7-6ad2-bfff-3ed4d56d714b'}}, tasks=(PregelTask(id='35c81b94-38dc-1f6b-5ec9-dab9d13a8ad1', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),)),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty! (get it?)', 'explanation': 'A classic play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it w

In [31]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06e591-188b-6ea2-8000-c15669419e53"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crusty" and had a lot of "filling" emotional issues to work through! (get it?)',
 'explanation': 'A deliciously clever joke. Let\'s break it down:\n\nThe joke is a play on words, using the characteristics of a samosa (a type of savory pastry) to create a pun. Here\'s how it works:\n\n1. **"Crusty"**: A samosa typically has a crispy, crusty exterior. In this joke, the word "crusty" is used to describe the samosa\'s emotional state, implying that it\'s feeling a little rough or hardened on the outside. However, "crusty" can also mean being irritable or gruff, which is a common reason for someone (or in this case, a samosa) to seek therapy.\n2. **"Filling" emotional issues**: A samosa is typically filled with a variety of ingredients, such as spiced potatoes, peas, or onions. In this joke, the word "filling" is used to describe the samosa\'s emotional problems, implying that it has a lot of

In [32]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crusty" and had a lot of "filling" emotional issues to work through! (get it?)', 'explanation': 'A deliciously clever joke. Let\'s break it down:\n\nThe joke is a play on words, using the characteristics of a samosa (a type of savory pastry) to create a pun. Here\'s how it works:\n\n1. **"Crusty"**: A samosa typically has a crispy, crusty exterior. In this joke, the word "crusty" is used to describe the samosa\'s emotional state, implying that it\'s feeling a little rough or hardened on the outside. However, "crusty" can also mean being irritable or gruff, which is a common reason for someone (or in this case, a samosa) to seek therapy.\n2. **"Filling" emotional issues**: A samosa is typically filled with a variety of ingredients, such as spiced potatoes, peas, or onions. In this joke, the word "filling" is used to describe the samosa\'s emotional problems, implying 

In [ ]:
#done.